# Laplacian path diagrams

I'd like an ALE/LPM started from a non-symmetric configuration of slits. It's not so easy to write down the map for the initial condition, but it should be possible numerically, using the Schwarz-Christoffel representation for "gearlike domains".

## Schwarz-Christoffel representation

The method comes from ["A Constructive Method for Numerically Computing Conformal Mappings for Gearlike Domains"](https://epubs.siam.org/doi/10.1137/0912013) by Kent Pearce, 1991.

In the paper, a *gearlike domain* is the region containing the origin (for us we will want it to contain $\infty$) bounded by a collection of arcs of circles centred at the origin and by segments of straight lines through the origin. Its logarithm is a periodic polygonal domain.

In the paper's notation, let $A$ be a gearlike domain in the complex plane with $n$ sides. Let $w_k$, $1 \leq k \leq n$, denote the vertices of $A$, some of which may lie at infinity (in our case they won't). For each finite vertex $w_k$ let $\pi \alpha_k$ denote the interior angle at $w_j$, and let $\pi \beta_k$ denote the exterior angle at $w_k$ (the definition of a gearlike domain limits us to $\beta_k \in \{-1,-1/2,0,1/2,1\}$).

Let $f$ be the conformal mapping $\mathbb{D} \to A$ with $f(0) = 0$, $f'(0) > 0$. Let $z_k := f^{-1}(w_k) \in \partial \mathbb{D}$.
Then the representation formula of Barnard and Pearce is $$\frac{z f'(z)}{f(z)} = \prod_{k=1}^n (1 - \bar z_k z)^{-\beta_k}.\tag{1}$$
We can "solve" this differential equation to write down "explicitly" $$f(z) = c z \exp\left( \int_0^z \frac{ -1 + \prod_{k=1}^n (1-\bar z_k w)^{-\beta_k}}{w}\mathrm{d}w \right).\tag{2}$$
So all we need to do is determine the preimages $z_k$ and the parameter $c$ (not so easy but can be done numerically), and then (this also isn't easy) compute that integral.

### Determining the preimages

If we rotate so that $z_n = 1$, then $c$ is also determined by the preimages. We can solve a system of non-linear equations to determine the remaining preimages.

Our domain takes a particular form. We'll invert at the very end, so the domain we want to map $\mathbb{D}$ into is the open unit disc with some slits removed. The slits are determined by their position and length, say $\theta_1, \dots, \theta_k$ and $d_1, \dots, d_k$. Each slit corresponds to three vertices: its tip and both sides of the base, so we have $n=3k$ vertices.

The _external_ angle at a tip is $-\pi$ (since the sum of the internal and external angles must be $\pi$). The external angle at each base is $\pi/2$ for the same reason.

Let the preimage of slit $j$'s tip be $z^0_{j}$, and the preimage of its bases be $z^{\pm}_j$.

Without loss of generality, $z^0_k = 1$ (if we're keeping the location of the last tip fixed, then this means we don't have $f'(0) > 0$ any more, but that's ok, we can just rotate at the end).
Then if $$w^* := \exp( \int_0^1 \frac{ -1 + \prod_{m=1}^n (1-\bar z_m w)^{-\beta_m}}{w}\mathrm{d}w )$$and $w_k$ is the location of the last tip, then we have $c = w_k/w^*$.

Pearce gets $n-2$ non-linear equations which we will solve. We _know_ the locations of the tips and bases $w_m$, and we are solving these equations to find the unknown values of $z_m$. For an _ansatz_ $(z_m)_{1 \leq m \leq n-1}$, we compute $\hat w_m = f(z_m)$ and $n-2$ of our equations are as follows for $1 \leq m \leq n-2$:
- If $|w_m| = |w_{m-1}|$ (this means that $w_m$ and $w_{m-1}$ are basepoints of adjacent slits), we have one equation $$(\arg(\hat w_m) - \arg(\hat w_{m-1})) - (\arg(w_m) - \arg(w_{m-1})) = 0.\tag{3c}$$ The right-hand side is the difference in argument between the two attachment points.
- If $\arg(w_m) = \arg(w_{m-1})$ (this is true if one of $w_m$ and $w_{m-1}$ is a tip so 2/3 of equations have this form), then we have the equation $$|\hat w_m| - |w_m| = 0.\tag{3a}$$
I believe that in all the above $w_0 := w_n$.

We add a final "equation" from $$\sum_{m=1}^n \beta_m \arg(z_m) \equiv 0 \; (\mathrm{mod}\; \pi).$$ I'm not sure how much trouble the discontinuity in this will cause.

#### Solving the non-linear equations

Following [Trefethen, 1979](https://www.proquest.com/docview/922000405?parentSessionId=IQLBX47O73EQ9fe%2Bxq01LKARr8cPF0pe%2Fd0FKNtAswo%3D&pq-origsite=primo&accountid=17230&sourcetype=Scholarly%20Journals), we can solve the non-linear system of equations by standard methods after transforming it to an unconstrained problem. Trefethen solved his non-linear equations with a package solver available at the time - I will follow his example and use Scipy's `scipy.optimize.fsolve`. It's important to carefully compute the integral involved in _evaluating_ $f : \mathbb{D} \to A$, since this determines how accurately we'll find the preimages of the bases and tips.

The code works as follows:

1. The points $(w_m)_{1 \leq m \leq n-1}$ are specified by the user.
2. We write a function `f(z,params)` computing $f(z ; z_1, ..., z_{n})$. This involves computing a singular integral numerically, including to calculate $c$ as a function of $(w_m)_m$.
3. Then we let $$y_m = \log\frac{\varphi_m - \varphi_{m-1}}{\varphi_{m+1}-\varphi_m},$$ for $1 \leq m \leq n-1$, where $\varphi_m := \arg(z_m)$ and $\varphi_0 := 0$. Note that these can take any value in $\mathbb{R}$. We write a function `get_params(ys)` that turns $(y_m)_{1 \leq m \leq n-1}$ back into $(z_m)_{1 \leq m \leq n-1}$.
4. We compute $(\hat w_m)_{1 \leq m \leq n-1}$ using `f` and $(z_m)_{1 \leq m \leq n-1}$.
5. We calculate the objective function, which returns a vector of length $n-1$. For $1 \leq m \leq n-2$, the $m$th entry is the left-hand side of $\mathrm{(3a)}$ if either of $w_m$ or $w_{m-1}$ is a tip, or $\mathrm{(3c)}$ otherwise. The final entry is $\sum_{m=1}^{n} \beta_m \arg(z_m) - i\pi$ where $i$ is chosen such that this value is in $(-\pi/2, \pi/2]$. I'm slightly worried about this one, but it might be ok.

In [ ]:
import numpy as np
from scipy.integrate import quad
from scipy.linalg import solve
from scipy.optimize import fsolve
from tqdm import trange
#from numba import jit

In [ ]:
def wstar(zs, betas):
    """
    Computes the w^* from Pearce, 1991.
    We can use this to find the value of c.
    
    zs    is a list of n points on the circle,
          with the last point equal to 1 + 0j;
    betas is a list of n exponents.

    This involves computing an integral numerically
    with a singularity potentially at one end of the range
    (in the centre of the disc).
    Since we chose a tip as w_n with z_n = 1 and beta_n = -1,
    the boundary has a zero instead of a pole.

    We can handle this using scipy.integrate.quad
    with weight='alg'.
    """
    def unweighted_integrand(t):
        prod = 1
        for i,z in enumerate(zs):
            prod *= (1 - z.conjugate()*t)**(-betas[i])
        return (prod - 1)/t
    integral_re = quad(lambda t : unweighted_integrand(t).real, 0, 1)
    integral_im = quad(lambda t : unweighted_integrand(t).imag, 0, 1)
    # print(f"Computed wstar with error value {integral[1]}")
    return np.exp(integral_re[0] + 1j*integral_im[0])

def ftip(preimage,zs,betas,c):
    """
    Computes the image of `preimage` when `preimage` is
    the preimage of a tip. It's the same method as when
    it's a base, but we do something a slightly different
    to deal with the pole for bases.
    """
    def unweighted_integrand(t):
        prod = 1
        for i,z in enumerate(zs):
            prod *= (1 - z.conjugate()*preimage*t)**(-betas[i])
        return (prod - 1)/t
    integral_re = quad(lambda t : unweighted_integrand(t).real, 0, 1)
    integral_im = quad(lambda t : unweighted_integrand(t).imag, 0, 1)
    return c*preimage*np.exp(integral_re[0] + 1j*integral_im[0])

def fbase(preimage_index,zs,betas,c):
    """
    Computes the image of `preimage` when `preimage` is
    the preimage of a base. Deals carefully with the pole
    at that base itself.
    We take the index instead of the value because then
    we can take the singular term out of the product and
    deal with it separately.
    """
    preimage = zs[preimage_index]
    def unweighted_integrand_1(t):
        # For t in (0,1/2]
        prod = 1
        for i in range(len(zs)):
            prod *= (1 - zs[i].conjugate()*preimage*t)**(-betas[i])
        return (prod - 1)/t
    def unweighted_integrand_2(t):
        # For t in [1/2, 1).
        # The other term becomes (1 - t)^{-1/2}.
        prod = 1
        for i in range(len(zs)):
            if i == preimage_index:
                continue
            else:
                prod *= (1 - zs[i].conjugate()*preimage*t)**(-betas[i])
        return prod/t
    integral1_re = quad(lambda t : unweighted_integrand_1(t).real,0,1/2)
    integral1_im = quad(lambda t : unweighted_integrand_1(t).imag,0,1/2)
    integral1 = integral1_re[0] + 1j*integral1_im[0]
    integral2_re = quad(lambda t : unweighted_integrand_2(t).real,1/2,1,weight='alg',wvar=(0,-1/2))
    integral2_im = quad(lambda t : unweighted_integrand_2(t).imag,1/2,1,weight='alg',wvar=(0,-1/2))
    integral2 = integral2_re[0] + 1j*integral2_im[0]
    integral = integral1 + integral2 - np.log(2)
    return c * preimage * np.exp( integral )

In [ ]:
def get_params(ys):
    """
    In the words of Trefethen, this is "easy to do, though not immediate".
    It involves defining a set of linear equations then solving them.
    """
    eys = np.exp(ys)
    # Construct a matrix
    A = np.zeros((len(ys),len(ys)))
    A[0,0] = -(1 + eys[0])
    A[0,1] = eys[0]
    for i in range(1,len(ys)-1):
        A[i,i-1] = 1
        A[i,i]   = -(1 + eys[i])
        A[i,i+1] = eys[i]
    A[len(ys)-1,len(ys)-2] = 1
    A[len(ys)-1,len(ys)-1] = -(1+eys[len(eys)-1])
    b = np.zeros(len(ys))
    b[len(ys)-1] = 2*np.pi*eys[len(eys)-1]
    phis = np.append(solve(A,b),0.0)
    zs = np.exp( 1j * phis )
    return zs

def get_w_hats(zs,betas,c):
    w_hats = np.empty(len(zs),dtype=np.complex128)
    preimage_type = 0 # 0 and 1 for bases, 2 for a tip.
    # The final wn is a tip, so the first two must be bases.
    for i in range(len(zs)):
        if preimage_type == 2:
            # We have a tip
            w_hats[i] = ftip(zs[i],zs,betas,c)
            preimage_type = 0
        else:
            # We have a base
            w_hats[i] = fbase(i,zs,betas,c)
            preimage_type += 1
    return w_hats

def my_arg(z):
    # argument in the range [0,2pi)
    theta = np.angle(z)
    if theta < 0:
        theta += 2*np.pi
    return theta

def objective_fn(ys,betas,ws):
    zs = get_params(ys)
    c = ws[-1]/wstar(zs,betas)
    w_hats = get_w_hats(zs,betas,c)
    equations = np.zeros(len(ys))
    preimage_type = 0 # 0 and 1 for bases, 2 for a tip.
    # The final wn is a tip, so the first two must be bases.
    # If preimage_type is 0 or 2 we have an equation like (3a),
    # and if preimage_type == 1 we have an equation like (3c).
    for i in range(len(ys)-1):
        if preimage_type == 0:
            # w_m is a base, w_{m-1} is a tip
            equations[i] = np.abs(w_hats[i]) - np.abs(ws[i])
            preimage_type += 1
        elif preimage_type == 1:
            # w_m is a base, w_{n-1} is also a base.
            equations[i] = my_arg(w_hats[i]/w_hats[i-1]) - my_arg(ws[i]/ws[i-1])
            preimage_type += 1
        else:
            # w_m is a tip, w_{m-1} is a base
            equations[i] = np.abs(w_hats[i]) - np.abs(ws[i])
            preimage_type = 0
    angle_sum = np.sum( [betas[i]*my_arg(zs[i]) for i in range(len(zs))] )
    remainder = angle_sum % np.pi
    if remainder > 0.5*np.pi:
        remainder = remainder - np.pi
    equations[len(ys)-1] = remainder
    return equations

### Set the parameters for $K_0$

In [ ]:
# Locations given in degrees (increasing order in [0,2pi)).
locations = [0,30,60,90,120,150,180,210,240,270,300,330]
# Lengths of the slits
lengths   = [1, 1, 1, 1,  1,  1,  1,  1,  1,  1,  1,  1]

In [ ]:
locations = [a*np.pi/180 for a in locations]

ws = np.empty(3*len(locations),dtype=np.complex128)
ws[0] = np.exp(-1j*locations[0])
for i in range(1,len(locations)):
    ws[3*i - 2] = np.exp(-1j*locations[i])
    ws[3*i - 1] = np.exp(-1j*locations[i]) / (1 + lengths[i])
    ws[3*i]     = np.exp(-1j*locations[i])
ws[3*len(locations)-2] = np.exp(-1j*locations[0])
ws[3*len(locations)-1] = np.exp(-1j*locations[0]) / (1 + lengths[0])

betas = np.empty(3*len(locations))
preimage_type = 0 # 0 and 1 are bases, 2 is a tip.
for i in range(3*len(locations)):
    if preimage_type == 2:
        betas[i] = -1
        preimage_type = 0
    else:
        betas[i] = 1/2
        preimage_type += 1

# Get the parameters.
ys = fsolve(objective_fn,np.zeros(len(ws)-1),args=(betas,ws))
zs = get_params(ys)
c = ws[-1] / wstar(zs,betas)
print(objective_fn(ys,betas,ws))

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.patches import Circle
from datetime import datetime
fig, ax = plt.subplots()
unit_circle = Circle((0,0),1,fill=False)
ax.add_patch(unit_circle)
# for z in zs:
#     plt.scatter(z.real,z.imag,marker='x',color='red')
## Draw the envelope
sigma = 0.00001
envelope = np.exp( 1j * np.linspace(0,2*np.pi,1000) - sigma )
for i in range(len(envelope)):
    envelope[i] = 1/ftip(envelope[i],zs,betas,c)
    plt.scatter(envelope[i].real, envelope[i].imag, marker='.', color='blue',s=2)
plt.gca().set_aspect('equal')
plt.show()

## Simulating the ALE

Now we can evaluate our map $\Phi_0$ and its derivative $\Phi_0'$. We have an explicit expression for the individual particle map $f_{\theta, c}$ and its derivative, so we have everything we need to simulate the ALE.

### Working in the exterior disc

One slight technicality is that we want the map for $\Delta = (\mathbb{C}_\infty \setminus \mathbb{D})^\circ$ instead of $\mathbb{D}$.
But in this case we just set $\Phi_0(z) = 1/f(1/z)$ and the Schwarz-Christoffel representation becomes $$\frac{z\Phi_0'(z)}{\Phi_0(z)} = \prod_{m=1}^n (z - \bar z_m)^{-\beta_m}.$$

In [ ]:
# All of ws, zs, betas and c are global variables.
def phi0(z):
    # Takes z in the exterior disc
    # to its image under phi0.
    return 1/ftip(1/z,zs,betas,c)

#@jit(nopython=True)
def phi0_prime(z,image):
    # Does it make more sense when eta > 0 to compute the
    # reciprocal of the derivative instead?
    derivative = image/z
    for m in range(len(zs)):
        derivative *= (z - zs[m].conjugate())**(-betas[m])
    return derivative

#@jit(nopython=True)
def phi0_prime_reciprocal(z,image):
    output = z/image
    for m in range(len(zs)):
        output *= (z - zs[m].conjugate())**(betas[m])
    return output

#@jit(nopython=True)
def slitmap(z,theta,capacity):
    ec = np.exp(capacity)
    w = z * np.exp(-1j*theta)
    return np.exp(1j*theta) * ((0.5*ec/w)*(w+1)**2 * (1 + np.sqrt((w*w + 2*(1 - 2/ec)*w + 1)/((w+1)**2))) - 1)

#@jit(nopython=True)
def slitmap_derivative(z,theta,capacity,fz):
    cosbetac = np.cos(np.asin(2*np.exp(-capacity)*np.sqrt(np.exp(capacity)-1)))
    w = z*np.exp(-1j*theta)
    return (fz/z) * (w-1) / np.sqrt(w*w - 2*cosbetac*w + 1)

#@jit(nopython=True)
def slitmap_derivative_reciprocal(z,theta,capacity,fz):
    cosbetac = np.cos(np.asin(2*np.exp(-capacity)*np.sqrt(np.exp(capacity)-1)))
    w = z*np.exp(-1j*theta)
    return (z/fz) * np.sqrt(w*w - 2*cosbetac*w + 1) / (w-1)

### Set the parameters for the ALE

In [ ]:
# Configuration
T = 2.0
n = 250
eta = 2.5
sigma = (T/n)**3

circle_pts = 1000

# Derived quantities
capacity = T/n
ec = np.exp(capacity)
d = 2*(ec - 1) + 2*ec*np.sqrt(1 - 1/ec)

In [ ]:
angle_seq = np.empty(shape=n)
raw_candidates = np.linspace(0,2*np.pi,num=circle_pts,endpoint=False)
for t in trange(n):
    # We rotate the candidate points through a random angle
    # to avoid evaluating the derivative exactly at the tips,
    # since when eta > 0 that would mean dividing by 0.
    candidates = (raw_candidates + 2*np.pi*np.random.random()) % (2*np.pi)
    z_candidates = np.exp( sigma + 1j*candidates )
    images = np.empty(shape=len(candidates),dtype=np.complex128)
    derivs = np.ones(shape=len(candidates))
    for i in range(len(candidates)):
        images[i] = z_candidates[i]
        for s in reversed(range(t)):
            new_loc = slitmap(images[i],angle_seq[s],capacity)
            derivs[i] *= np.abs(slitmap_derivative_reciprocal(images[i],angle_seq[s],capacity,new_loc))
            images[i] = new_loc
        derivs[i] *= np.abs(phi0_prime_reciprocal(images[i],phi0(images[i])))
    derivs = np.nan_to_num(derivs**(eta))
    chosen_index = np.random.choice(circle_pts,p=derivs/sum(derivs))
    angle_seq[t] = candidates[chosen_index]

In [ ]:
# Let's draw the picture!
from colorspace import hcl_palettes
my_palette = hcl_palettes().get_palette(name="Batlow")
colours = my_palette.colors(n)

slitdensity = 100 # points plotted per slit.
slit = np.linspace(1+d,1,num=slitdensity,endpoint=False)

cluster = np.empty(shape=slitdensity*n,dtype=np.complex128)
for t in range(n):
    cluster[:t*slitdensity] = slitmap(cluster[:t*slitdensity],angle_seq[n-1-t],capacity)
    cluster[t*slitdensity:(t+1)*slitdensity] = np.exp(1j*angle_seq[n-1-t])*slit
for i in trange(len(cluster)):
    cluster[i] = phi0(cluster[i])

fig, ax = plt.subplots()
ax.set_aspect(1.0)
unit_circle = Circle((0,0),1,fill=False)
ax.add_patch(unit_circle)
for t in range(n):
    ax.scatter(cluster.real[t*slitdensity:(t+1)*slitdensity],cluster.imag[t*slitdensity:(t+1)*slitdensity],
               color=colours[t],s=1)
for k in range(len(locations)):
    arm = np.exp(1j*locations[k]) * np.linspace(1+lengths[k],1,num=slitdensity,endpoint=False)
    ax.scatter(arm.real,arm.imag,color='k',s=1)

plt.show()
now = datetime.now().isoformat()
fig.savefig(f"{now}.pdf")